In [1]:
library(dplyr)
library(ggplot2)
library(readr)
library(reshape2)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: lattice

Loading required package: grid

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [2]:
df <- read_csv('../Datasets/nyserda-electric-vehicle-drive-clean-rebate-data-beginning-2017-1.csv')

Rows: 23174 Columns: 11
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (7): Data through Date, Submitted Date, Make, Model, County, EV Type, Tr...
dbl (4): ZIP, Annual GHG Emissions Reductions (MT CO2e), Annual Petroleum Re...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
set.seed(42)

In [4]:
df

Data through Date,Submitted Date,Make,Model,County,ZIP,EV Type,Transaction Type,Annual GHG Emissions Reductions (MT CO2e),Annual Petroleum Reductions (gallons),Rebate Amount (USD)
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
02/29/2020,01/13/2018,Ford,Focus,Westchester,10598,BEV,Lease,2.76,592.89,1700
02/29/2020,09/25/2017,Ford,Focus,Oneida,13601,BEV,Lease,2.76,592.89,1700
02/29/2020,04/26/2018,Ford,Focus,Suffolk,11776,BEV,Lease,2.76,592.89,1700
02/29/2020,08/30/2017,Ford,Focus,Nassau,11756,BEV,Lease,2.76,592.89,1700
02/29/2020,05/29/2018,Ford,Focus,Onondaga,13039,BEV,Purchase,2.76,592.89,1700
02/29/2020,07/10/2017,Ford,Focus,Westchester,10514,BEV,Purchase,2.76,592.89,1700
02/29/2020,10/02/2017,Ford,Focus,Richmond,10312,BEV,Lease,2.76,592.89,1700
02/29/2020,12/27/2017,Ford,Focus,Dutchess,12538,BEV,Purchase,2.76,592.89,1700
02/29/2020,03/12/2018,Ford,Focus,Rockland,10965,BEV,Lease,2.76,592.89,1700


In [5]:
df <- select(df, -c("Data through Date", "ZIP"))

In [6]:
# Assign new names directly to the specific columns
names(df)[names(df) == "Annual GHG Emissions Reductions (MT CO2e)"] <- "Annual CO2 Reductions"
names(df)[names(df) == "Annual Petroleum Reductions (gallons)"] <- "Annual Petroleum Reductions"

In [7]:

cols <- c('Annual CO2 Reductions', 'Annual Petroleum Reductions',	'Rebate Amount (USD)')

df[cols] <- sapply(df[cols], as.numeric)

for (col in cols) {
  quartiles <- quantile(df[[col]], c(0.1, 0.7))
  q1 <- quartiles[1]
  q3 <- quartiles[2]

  low_bound <- q1 - 1.5 * (q3 - q1)
  upp_bound <- q3 + 1.5 * (q3 - q1)

  df <- df[df[[col]] >= low_bound & df[[col]] <= upp_bound, ]
}

In [8]:
df

Submitted Date,Make,Model,County,EV Type,Transaction Type,Annual CO2 Reductions,Annual Petroleum Reductions,Rebate Amount (USD)
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
01/13/2018,Ford,Focus,Westchester,BEV,Lease,2.76,592.89,1700
09/25/2017,Ford,Focus,Oneida,BEV,Lease,2.76,592.89,1700
04/26/2018,Ford,Focus,Suffolk,BEV,Lease,2.76,592.89,1700
08/30/2017,Ford,Focus,Nassau,BEV,Lease,2.76,592.89,1700
05/29/2018,Ford,Focus,Onondaga,BEV,Purchase,2.76,592.89,1700
07/10/2017,Ford,Focus,Westchester,BEV,Purchase,2.76,592.89,1700
10/02/2017,Ford,Focus,Richmond,BEV,Lease,2.76,592.89,1700
12/27/2017,Ford,Focus,Dutchess,BEV,Purchase,2.76,592.89,1700
03/12/2018,Ford,Focus,Rockland,BEV,Lease,2.76,592.89,1700


In [9]:
df$'Submitted Month '<- sapply(strsplit(as.character(df$'Submitted Date'), "/"), function(x) x[1])
df$'Submitted Year' <- sapply(strsplit(as.character(df$'Submitted Date'), "/"), function(x) x[3])


df <- df[, !names(df) %in% c("Submitted Date", "Submitted Month")]

In [ ]:
cols <- c("Make", "Model", "EV Type","County","Transaction Type", "Rebate Amount (USD)", "Annual CO2 Reductions",	"Annual Petroleum Reductions")

for (column in cols) {
  df[[column]] <- as.numeric(factor(df[[column]]))
}

In [ ]:

columns <- c("Annual CO2 Reductions", "Annual Petroleum Reductions")

df[columns] <- scale(df[columns])

In [ ]:
write_csv(df, '../Cleaned Datasets R/rebate_cleaned.csv')